In [7]:
import polars as pl

file_path = "../../data/letterboxd_filtered_0.99.jsonl"

try:
    df = pl.read_ndjson(file_path)
    print("Successfully loaded!\n")
except Exception as e:
    print(f"Error loading file: {e}\n")
    exit(1)

Successfully loaded!



In [8]:
import json

num_films = 0
num_synopses = 0
num_reviews = 0

synopsis_chars = 0
review_chars = 0

with open(file_path, encoding="utf-8") as f:
    for line in f:
        num_films += 1
        data = json.loads(line)

        # Synopsis
        synopsis = data.get("synopsis", "")
        if synopsis:
            num_synopses += 1
            synopsis_chars += len(synopsis)

        # Reviews
        reviews = data.get("review_texts", [])
        num_reviews += len(reviews)
        for r in reviews:
            review_chars += len(r)

total_chars = synopsis_chars + review_chars

print("Films:", num_films)
print("Synopses:", num_synopses)
print("Reviews:", num_reviews)
print("Synopsis characters:", synopsis_chars)
print("Review characters:", review_chars)
print("Total characters:", total_chars)

Films: 162823
Synopses: 161179
Reviews: 289669
Synopsis characters: 44248874
Review characters: 59089167
Total characters: 103338041


In [7]:
print(f"Number of rows: {len(df):,}")
print(f"Memory size: {df.estimated_size('mb'):.2f} MB")


null_counts = df.null_count()
print("\nNumber of NULL values per column:")
for col in df.columns:
    null_count = null_counts[col][0]
    percentage = (null_count / len(df)) * 100
    print(f"  {col:20s}: {null_count:6d} ({percentage:5.2f}%)")

Number of rows: 262,842
Memory size: 352.70 MB

Number of NULL values per column:
  title               :      0 ( 0.00%)
  year                :      0 ( 0.00%)
  synopsis            :   4208 ( 1.60%)
  review_texts        :      0 ( 0.00%)


In [4]:
df_null_values = df.filter(pl.col("synopsis").is_null()).select("title").unique()

print(f"Number of different movies with NULL synopsis: {len(df_null_values)}")
print(df_null_values.head(10))

Number of different movies with NULL synopsis: 15424
shape: (10, 1)
┌─────────────────────────────────┐
│ title                           │
│ ---                             │
│ str                             │
╞═════════════════════════════════╡
│ Bastille: MTV Unplugged         │
│ Élémentaire                     │
│ El equipo de mi Barrio          │
│ Uma Versão Minha Que Já Não Ex… │
│ Les Aventures du Nexus VI       │
│ Internships                     │
│ Lumi Videla                     │
│ Ombr                            │
│ NWA WrestleWar 1989             │
│ Confrontación                   │
└─────────────────────────────────┘


In [5]:
interesting_cols = ["synopsis", "review_text"]
for col in interesting_cols:
    words = df.filter(pl.col(col).is_not_null()).select(pl.col(col).str.split(" ").list.len().alias("word_count"))

    stats = words.select(
        pl.col("word_count").min().alias("min"),
        pl.col("word_count").max().alias("max"),
        pl.col("word_count").mean().alias("mean"),
        pl.col("word_count").sum().alias("total"),
    )

    print(f"\n{col}:")
    print(stats)


synopsis:
shape: (1, 4)
┌─────┬─────┬───────────┬──────────┐
│ min ┆ max ┆ mean      ┆ total    │
│ --- ┆ --- ┆ ---       ┆ ---      │
│ u32 ┆ u32 ┆ f64       ┆ u32      │
╞═════╪═════╪═══════════╪══════════╡
│ 1   ┆ 203 ┆ 46.926372 ┆ 92896057 │
└─────┴─────┴───────────┴──────────┘

review_text:
shape: (1, 4)
┌─────┬─────┬───────────┬──────────┐
│ min ┆ max ┆ mean      ┆ total    │
│ --- ┆ --- ┆ ---       ┆ ---      │
│ u32 ┆ u32 ┆ f64       ┆ u32      │
╞═════╪═════╪═══════════╪══════════╡
│ 1   ┆ 166 ┆ 34.523416 ┆ 70008965 │
└─────┴─────┴───────────┴──────────┘


In [6]:
short_synopsis = (
    df.filter(pl.col("synopsis").is_not_null() & (pl.col("synopsis").str.split(" ").list.len() <= 10))
    .select(["title", "synopsis", "review_text"])
    .unique(subset=["title"])
)

print(f"Movies with short synopses: {len(short_synopsis)}")
for row in short_synopsis.head().iter_rows(named=True):
    print(f"\nTitle: {row['title']}")
    print(f"Synopsis: {row['synopsis']}")
    print("-" * 80)

Movies with short synopses: 16683

Title: Sound of Silence
Synopsis: Evil comes with sound. So better be quiet.
--------------------------------------------------------------------------------

Title: Johnny Vik
Synopsis: Johnny plays drums at midnight, but his survival is tentative.
--------------------------------------------------------------------------------

Title: To Cross the Line
Synopsis: Inspector Vladimir Sazhin investigates an unusual case.
--------------------------------------------------------------------------------

Title: Othello
Synopsis: Filmed version of the 2019 Stratford Festival production.
--------------------------------------------------------------------------------

Title: Strawberry Shortcake: Her Very First Adventures
Synopsis: Original episodes of Strawberry Shortcake
--------------------------------------------------------------------------------


In [7]:
short_review = df.filter(
    pl.col("review_text").is_not_null() & (pl.col("review_text").str.split(" ").list.len() <= 10)
).select(["title", "synopsis", "review_text"])

print(f"Movies with short review: {len(short_review)}")
for row in short_review.head().iter_rows(named=True):
    print(f"\nTitle: {row['title']}")
    print(f"Review: {row['review_text']}")
    print("-" * 80)

Movies with short review: 589765

Title: Come and See
Review: What a horrible nightmare!
--------------------------------------------------------------------------------

Title: Come and See
Review: (guy who's still buzzing from Spider-Man: Across the Spider-Verse)
--------------------------------------------------------------------------------

Title: Come and See
Review: Playing this at grandma’s bingo party next week.
--------------------------------------------------------------------------------

Title: Seven Samurai
Review: too many sweaty ass cheeks, 5 stars
--------------------------------------------------------------------------------

Title: Seven Samurai
Review: this is what the avengers think they look like
--------------------------------------------------------------------------------
